In [1]:
import cv2
import numpy as np
import os
import glob
import time
from IPython.display import clear_output
import json

In [2]:
cap = cv2.VideoCapture(1)
#set autofocus to auto
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
ret, frame = cap.read()
save_path = 'calibration_images'
img_counter = 0
#checkerboars settings
CHECKERBOARD = (5,5)
do_wait = False
while True:
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)
        result, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if result == True:qq
            cv2.imwrite(os.path.join(save_path, 'calibration_image_{}.png'.format(img_counter)), frame)
            img_counter += 1
            clear_output()
            #write the corners on the image
            cv2.drawChessboardCorners(frame, CHECKERBOARD, corners, result)
            do_wait = True
        cv2.imshow('frame', frame)
        if do_wait:
            do_wait = False
            cv2.waitKey(5000)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        #if the user presses 'c' then continue
        if cv2.waitKey(1) & 0xFF == ord('c'):
            continue
    #refocus the camera
    else:
        break

In [3]:
img_path = r'calibration_images'
imgs_path = glob.glob(img_path + '/*.png')
for img_path in imgs_path:
    print(img_path)

calibration_images\calibration_image_0.png
calibration_images\calibration_image_1.png
calibration_images\calibration_image_10.png
calibration_images\calibration_image_11.png
calibration_images\calibration_image_12.png
calibration_images\calibration_image_13.png
calibration_images\calibration_image_14.png
calibration_images\calibration_image_15.png
calibration_images\calibration_image_16.png
calibration_images\calibration_image_17.png
calibration_images\calibration_image_18.png
calibration_images\calibration_image_19.png
calibration_images\calibration_image_2.png
calibration_images\calibration_image_20.png
calibration_images\calibration_image_21.png
calibration_images\calibration_image_22.png
calibration_images\calibration_image_23.png
calibration_images\calibration_image_24.png
calibration_images\calibration_image_3.png
calibration_images\calibration_image_4.png
calibration_images\calibration_image_5.png
calibration_images\calibration_image_6.png
calibration_images\calibration_image_7.

In [4]:
# Defining the dimensions of checkerboard
CHECKERBOARD = (5,5)
CHECKERBOARD_CELL_SIZE = 0.015 # in meters
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

for fname in imgs_path:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        # print 'press any key to continue'
        img = cv2.putText(img, 'press any key to continue', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[651.93657606   0.         308.3220556 ]
 [  0.         653.12358509 244.76663015]
 [  0.           0.           1.        ]]
dist : 

[[ 1.47411414e-01 -9.95930889e-01 -1.00400073e-03  1.34558490e-03
   1.16540809e+00]]
rvecs : 

(array([[-0.56101633],
       [ 0.12291077],
       [ 0.50751701]]), array([[-0.39071083],
       [ 0.23277188],
       [ 0.77634715]]), array([[-0.24645317],
       [ 0.05330712],
       [ 0.03149622]]), array([[-0.06773636],
       [-0.33386441],
       [-3.11277276]]), array([[ 0.22615298],
       [-0.51817494],
       [-3.03503116]]), array([[-0.3676667 ],
       [-0.11355585],
       [ 0.01406423]]), array([[-0.09000306],
       [ 0.42966502],
       [ 3.08987864]]), array([[0.18183063],
       [0.51247373],
       [1.40914779]]), array([[0.17099853],
       [0.48730198],
       [1.40805932]]), array([[-0.85504158],
       [ 0.20930396],
       [ 0.12221823]]), array([[-0.43611089],
       [-0.95696879],
       [-1.94837527]]), array([

In [5]:
#save the camera matrix and distortion coefficients
with open('calibration_data.json', 'w') as outfile:
    json.dump({'mtx': mtx.tolist(), 'dist': dist.tolist()}, outfile)
#reconstruct the camera matrix and distortion coefficients
with open('calibration_data.json', 'r') as infile:
    reconstruction = json.load(infile)
    new_mtx = np.array(reconstruction['mtx'])
    new_dist = np.array(reconstruction['dist'])
    if np.allclose(new_mtx, mtx):
        print('mtx is the same')
    if np.allclose(new_dist, dist):
        print('dist is the same')

mtx is the same
dist is the same


In [6]:
cap.release()
cv2.destroyAllWindows()
print('done')

done
